In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference_cherry.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation:   0%|                             | 0/30 [00:00<?, ?it/s]/home/pk/Documents/cellbgnet/cellbgnet/utils/hardware.py:9: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  return torch.cuda.FloatTensor(x.astype('float32'))
Eval image generation: 100%|████████████████████| 30/30 [00:11<00:00,  2.65it/s]


eval images shape: (30, 1041, 1302) contain 18211 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
model.evaluation_params

{'number_images': 30,
 'use_cell_masks': True,
 'density_in_cells': 0.00625,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-24-CB4779/pool_CB4779_pos74/phase_cherry_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-24-CB4779/pool_CB4779_pos74/edt_noise.pkl',
 'eval_size': [1041, 1302],
 'molecules_per_img': 100,
 'margin_empty': 0.05,
 'candidate_threshold': 0.2,
 'nms_threshold': 0.5,
 'print_result': False,
 'batch_size': 16,
 'padded_background': 206.0,
 'eval_filenames': [PosixPath('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-24-CB4779/pool_CB4779_pos74/phase_cherry_mask/Pos74_img_000000129.png'),
  PosixPath('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-24-CB4779/pool_CB4779_pos74/phase_cherry_mask/Pos74_img_000000082.png'),
  PosixPath('/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-24-CB4779/pool_CB4779_pos74/phase_cherry_mask/Pos74_img_000000122.png'),
  PosixPath('/mnt/sda

In [10]:
start_time = time.time()
model.fit(batch_size=64, max_iters=10000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 912.967 || Time Upd.: 780.6 ms  || BatchNr.: 100
 || Cost: 225.031 || Time Upd.: 773.1 ms  || BatchNr.: 200
 || Cost: 127.903 || Time Upd.: 775.5 ms  || BatchNr.: 300
 || Cost: 120.498 || Time Upd.: 775.1 ms  || BatchNr.: 400
 || Cost: 97.279 || Time Upd.: 778.7 ms  || BatchNr.: 500
 || Cost: 83.913 || Time Upd.: 778.1 ms  || BatchNr.: 600
 || Cost: 84.396 || Time Upd.: 781.2 ms  || BatchNr.: 700
 || Cost: 74.441 || Time Upd.: 774.1 ms  || BatchNr.: 800
 || Cost: 74.352 || Time Upd.: 782.0 ms  || BatchNr.: 900
 || Cost: 70.104 || Time Upd.: 785.4 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 18211FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 18204 ,preds: 10170
JoR: 1.000 || Eff_3d: 24.396 || Jaccard: 0.556 || Factor: 457.872 || RMSE_lat: 55.643 || RMSE_ax: 133.201 || RMSE_x: 35.6